<a href="https://colab.research.google.com/github/LarrySnyder/stockpyl/blob/master/notebooks/with_solutions/Stockpyl_Tutorial_%C2%A74_Multi_Echelon_Inventory_Optimization_(MEIO)_WITH_SOLUTIONS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Stockpyl Tutorial
=================

(This notebook is a companion to Snyder, L. V., "[Stockpyl: A Python Package for Inventory Optimization and Simulation](https://pubsonline.informs.org/doi/10.1287/educ.2023.0256)," in: Bish, E. K. and H. Balasubramanian, _INFORMS TutORials in Operations Research_, 156–197, 2023.)



# Section 4: Multi-Echelon Inventory Optimization (MEIO)

Multi-echelon inventory optimization (MEIO) is significantly more difficult than single-echelon problems, primarily because a given node or stage (we use the terms interchangeably) experiences stochastic lead times from its upstream nodes, even if the transportation lead time is deterministic, due to random stockouts at the upstream nodes. Moreover, the probability distribution of the lead time depends on the base-stock levels (or other inventory policy parameters) at the upstream nodes, but the optimal upstream base-stock levels depend on the optimal downstream base-stock levels, which in turn depend on the probability distribution of the lead times. This vicious cycle makes it difficult even to calculate the expected cost of a given set of base-stock levels, let alone optimize them.

Two primary types of models have been developed to deal with this issue. _Stochastic-service models_ (SSM) assume that each stage meets the demand from stock whenever possible and experiences stockouts otherwise, much like the newsvendor and other single-stage problems. The upstream lead times will be stochastic due to stockouts, and the expected cost function must account for these lead times either exactly or approximately, leading to considerable challenges in both formulating the expected cost and optimizing it. This is the approach taken in the seminal paper by Clark and Scarf (1960) and many subsequent works.

In contrast, _guaranteeed-service models_ (GSM) assume that each stage provides a deterministic upper bound on its outbound lead time. In particular, each stage sets a _committed service time_ (CST) and guarantees that it will satisfy every demand within that CST. This allows a more tractable formulation of the MEIO problem since the upstream lead times are now deterministic. On the other hand, the tractability comes at an expense, in the form of a relatively strong assumption that the customer demand is bounded. There is a one-to-one relationship between a set of CSTs, one for each node, and a set of corresponding base-stock levels. Therefore, if we think of SSM as a more accurate model of real-world inventory systems (a claim that is open to debate, see, e.g., Graves and Willems 2003), the GSM approach can be viewed as a heuristic for setting base-stock levels in an SSM system. The GSM approach was pioneered by Kimball (1988) and Simpson (1958), and brought into more widespread use by Graves and Willems (2000).



Stockpyl contains code to solve the following types of multi-echelon inventory optimization (MEIO) problems:

* Serial systems under the SSM (`ssm_serial` module; see Section 4.3)
* Serial or tree systems under the GSM (`gsm_serial` and `gsm_tree` modules; see Section 4.4)
* SSM systems with arbitrary topology, optimized using enumeration or coordinate descent (`meio_general` module; see Section 4.5)


Before discussing these modules, we discuss an important data type used throughout them.




First, install the package:

In [2]:
!pip install stockpyl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.3/146.3 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.4/526.4 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 575.5/575.5 kB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.0/127.0 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.9/112.9 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.3/99.3 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 398.5/398.5 kB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.7/526.7 kB 40.0 MB/s eta 0:00:00
INFO: pip is lo

## 4.1 The `SupplyChainNetwork` and `SupplyChainNode` Classes

The MEIO code in Stockpyl makes use of the [`SupplyChainNetwork`](https://stockpyl.readthedocs.io/en/latest/api/datatypes/supply_chain_network.html#stockpyl.supply_chain_network.SupplyChainNetwork) class, which contains all of the data for an MEIO instance. For some functions, you provide data in the form of lists, dictionaries, or singletons and the function builds the `SupplyChainNetwork` object for you, while other functions require you to pass a `SupplyChainNetwork` object directly.

A `SupplyChainNetwork`, in turn, consists of one or more [`SupplyChainNode`](https://stockpyl.readthedocs.io/en/latest/api/datatypes/supply_chain_node.html#stockpyl.supply_chain_node.SupplyChainNode) objects. When you create a `SupplyChainNode`, you provide an index and any parameters you wish. For example:

In [3]:
from stockpyl.supply_chain_node import SupplyChainNode
# Build a node.
my_node = SupplyChainNode(
    index=1,
    local_holding_cost=3,
    stockout_cost=20,
    shipment_lead_time=2
)
# Build another node.
my_other_node = SupplyChainNode(
    index=2,
    name="other_node",
    echelon_holding_cost=1.5
)

(A list of all available parameters is in the [`online documentation`](https://stockpyl.readthedocs.io/en/latest/api/datatypes/supply_chain_node.html#stockpyl.supply_chain_node.SupplyChainNode) for the `SupplyChainNode` class.)

You can then add these nodes to a `SupplyChainNetwork`:

In [4]:
from stockpyl.supply_chain_network import SupplyChainNetwork
# Create an empty supply chain network.
network = SupplyChainNetwork()
# Add the first node to it.
network.add_node(my_node)
# Add the second node by adding it as a successor to the first node.
network.add_successor(my_node, my_other_node)

The `network` object now contains both nodes, and a (directed) edge from `my_node` to `my_other_node`. Nodes can be accessed either from a `SupplyChainNetwork` object's `nodes` attribute (a list of nodes), or using its member function `get_node_from_index()`:

In [5]:
n1 = network.nodes[0]
n1.index

1

In [6]:
n1.local_holding_cost

3

In [7]:
n2 = network.get_node_from_index(2)
n2.index

2

In [8]:
n2.echelon_holding_cost

1.5

The [`supply_chain_network`](https://stockpyl.readthedocs.io/en/latest/api/datatypes/supply_chain_network.html#module-stockpyl.supply_chain_network) module contains functions for quickly building certain types of multi-echelon networks.
For example, [`single_stage_system()`](https://stockpyl.readthedocs.io/en/latest/api/datatypes/supply_chain_network.html#stockpyl.supply_chain_network.single_stage_system) generates a single-node network:

In [9]:
from stockpyl.supply_chain_network import single_stage_system
network = single_stage_system(holding_cost=0.18,
    stockout_cost=0.70,
    demand_type='N',
    mean=50, standard_deviation=8,
    policy_type='BS',
    base_stock_level=56.6
)
network.nodes[0].stockout_cost

0.7

The [`serial_system()`](https://stockpyl.readthedocs.io/en/latest/api/datatypes/supply_chain_network.html#stockpyl.supply_chain_network.serial_system) function generates a serial system. Its signature is
```python
serial_system(num_nodes, node_order_in_system=None, node_order_in_lists=None,
    **kwargs)
```
The `node_order_in_system` parameter indicates the order of the node indices in the serial system; if omitted, the nodes are assumed to be indexed 0, ..., `num_nodes`-1, with node 0 upstream and node `num_nodes`-1 downstream. The `node_order_in_lists` parameter allows you to provide data in a different order if you wish. The `kwargs` parameters specify the attributes (data) for the nodes in the network. If they are provided, they must be either a dictionary, a list, or a singleton, with the following requirements:
* If the parameter is a dictionary, then the keys must contain the node indices and the values must contain the corresponding attribute values. If a given node index is contained in `node_order_in_system` (or in 0, ..., `num_nodes`-1, if `node_order_in_system` is not provided) but is not a key in the dictionary, the attribute value is set to `None` for that node.
* If the parameter is a singleton, then the attribute is set to that value for all nodes.
* If the parameter is a list and `node_order_in_lists` is provided, `node_order_in_lists` must contain the same indices as `node_order_in_system` (if it is provided) or 0, ..., `num_nodes`-1 (if it is not). The values in the list are assumed to correspond to the node indices in the order they are specified in `node_order_in_lists`. That is, the value in slot `k` in the parameter list is assigned to the node with index `node_order_in_lists[k]`.
* If the parameter is a list and `node_order_in_lists` is not provided, the values in the list are assumed to correspond to nodes in the same order as `node_order_in_system` (or in 0, ..., `num_nodes`-1, if `node_order_in_system` is not provided).

This scheme is meant to provide maximum flexibility, but it can also be confusing at first, so we recommend testing the network you build by querying it to double-check, as we do in the example below.

**Example:** Build a 3-node serial system, with nodes labeled $2 \to 1 \to 0$. The local holding costs at nodes 0, 1, 2 are 7, 4, 2, respectively. The demand at node 0 is distributed as $N(10,2^2)$. The node uses a base-stock policy with a base-stock level of 5.

<center><img src="https://github.com/LarrySnyder/stockpyl/raw/master/notebooks/images/tutorial_3_stage_system.png" width="400"/></center>


In [10]:
from stockpyl.supply_chain_network import serial_system
# Build a 3-node serial system.
network = serial_system(
    num_nodes=3,
    node_order_in_system=[2, 1, 0],
    node_order_in_lists=[0, 1, 2],
    local_holding_cost=[7, 4, 2],
    demand_type='N',
    mean=10,
    standard_deviation=2,
    policy_type='BS',
    base_stock_level=5
)
# Check the index of the (only) source node.
network.source_nodes[0].index

2

In [11]:
# Check the holding cost of the middle node (source node's successor).
network.source_nodes[0].successors()[0].local_holding_cost

4

In this case, although the network consists of nodes in the order $2 \to 1 \to 0$, the `local_holding_cost` parameter is provided as a list with node 0 first, then node 1, then node 2.

Once the network is constructed, we can query in various ways. In the code above, `network.source_nodes[0]` gets the first (actually, only) source node (i.e., a node with no upstream predecessors) and `succesors()` gets a list of that node's successor nodes.

---
**Exercise:** Construct a 2-node serial system in which node 1 is upstream and node 2 is downstream (that is, node 2 is node 1's successor). The local holding costs at nodes 1 and 2 are 30 and 40, respectively. The lead times at nodes 1 and 2 are 6 and 2, respectively. Query your network to make sure that it is correct.

In [12]:
## SOLUTION
network = serial_system(
    num_nodes=2,
    node_order_in_system=[1, 2],
    local_holding_cost=[30, 40],
    lead_time=[6, 2],
    standard_deviation=12
)
# Check the index of the (only) source node.
network.source_nodes[0].index

1

In [13]:
## SOLUTION
# Check the index of the (only) sink node.
network.sink_nodes[0].index

2

In [14]:
## SOLUTION
# Check that node 1 is node 2's predecessor, and node 2 is node 1's succesor.
print(f'Node 2\'s predecessor is node {network.get_node_from_index(2).predecessor_indices()[0]}')
print(f'Node 1\'s successor is node {network.get_node_from_index(1).successor_indices()[0]}')

Node 2's predecessor is node 1
Node 1's successor is node 2


In [15]:
## SOLUTION
# Check holding costs and lead times.
node1 = network.get_node_from_index(1)
node2 = network.get_node_from_index(2)
print(f'Node 1\'s holding cost is {node1.local_holding_cost}')
print(f'Node 2\'s holding cost is {node2.local_holding_cost}')
print(f'Node 1\'s lead time is {node1.lead_time}')
print(f'Node 2\'s lead time is {node2.lead_time}')


Node 1's holding cost is 30
Node 2's holding cost is 40
Node 1's lead time is 6
Node 2's lead time is 2


---
Similarly, the [`owmr_system()`](https://stockpyl.readthedocs.io/en/latest/api/datatypes/supply_chain_network.html#stockpyl.supply_chain_network.owmr_system) and [`mwor_system()`](https://stockpyl.readthedocs.io/en/latest/api/datatypes/supply_chain_network.html#stockpyl.supply_chain_network.mwor_system) functions allow you to quickly build one-warehouse, multi-retailer (OWMR) and multi-warehouse, one-retailer (MWOR) systems.

Finally, the [`network_from_edges()`](https://stockpyl.readthedocs.io/en/latest/api/datatypes/supply_chain_network.html#stockpyl.supply_chain_network.network_from_edges) function allows you build a network with arbitrary topology by providing a list of the edges in the network, along with the data for each node. The function has the signature
```python
network_from_edges(edges, node_order_in_lists=None, **kwargs)
```
The `edges` parameter is a list of edges, with each edge specified as a tuple `(a, b)`, where `a` is the index of the predecessor node and `b` is the index of the successor node. If `edges` is `None` or an empty list, a single-node network is created. The `node_order_in_lists` parameter works similarly to that in the `serial_system()` function, described above. The `kwargs` parameters are handled similarly to `serial_system()` as well, with the modification that indices are matched to the indices in the `edges` list rather than to `node_order_in_system` or 0, ..., `num_nodes` - 1.

**Example:** The following code creates a network that represents the system depicted below. The holding costs at nodes 1–4 are 4, 7, 2, 1, respectively, and the stockout costs at nodes 1 and 2 are 20 and 50, respectively.

<center><img src="https://github.com/LarrySnyder/stockpyl/raw/master/notebooks/images/tutorial_4_stage_system.png" width="400"/></center>

In [16]:
from stockpyl.supply_chain_network import network_from_edges
# Create a network by specifying the edges.
network1 = network_from_edges(
    edges=[(3, 1), (3, 2), (4, 1)],
    node_order_in_lists=[1, 2, 3, 4],
    local_holding_cost=[4, 7, 2, 1],
    stockout_cost=[20, 50, None, None],
    demand_type=['N', 'N', None, None],
    mean=[10, 15, None, None],
    standard_deviation=[2, 4, None, None]
)
# Print some details about each node in the network.
for node in network1.nodes:
    print(f"Node {node.index}: h = {node.local_holding_cost} p = {node.stockout_cost} mu = {node.demand_source.mean}  sigma = {node.demand_source.standard_deviation}")

Node 3: h = 2 p = None mu = None  sigma = None
Node 1: h = 4 p = 20 mu = 10  sigma = 2
Node 2: h = 7 p = 50 mu = 15  sigma = 4
Node 4: h = 1 p = None mu = None  sigma = None


Alternatively, we could construct the same network by constructing each node individually and adding them to the network, specifying the appropriate predecessors and successors. In this case, we need to explicitly set the `demand_source` attribute for the sink nodes (nodes that face external demand) and the `supply_type` attribute for the source nodes (nodes that have external supply). The `demand_source` is set to a [`DemandSource`](https://stockpyl.readthedocs.io/en/latest/api/datatypes/demand_source.html#stockpyl.demand_source.DemandSource) object; in particular, the demand is normally distributed at nodes 1 and 2, with different means and standard deviations. (See the online documentation for more information.) The `supply_type` attribute is set to `'U'` for nodes 3 and 4 to indicate unlimited supply. (This is currently the only type of supply supported in the `SupplyChainNode` class.)

**Example:** The following code creates the same network as in the previous example, but building it node-by-node:

In [17]:
from stockpyl.supply_chain_network import SupplyChainNetwork
from stockpyl.supply_chain_node import SupplyChainNode
from stockpyl.demand_source import DemandSource
# Build 4 nodes.
node1 = SupplyChainNode(index=1, local_holding_cost=4, stockout_cost=20,
	demand_source=DemandSource(type='N', mean=10, standard_deviation=2))
node2 = SupplyChainNode(index=2, local_holding_cost=7, stockout_cost=50,
	demand_source=DemandSource(type='N', mean=15, standard_deviation=4))
node3 = SupplyChainNode(index=3, local_holding_cost=2, supply_type='U')
node4 = SupplyChainNode(index=4, local_holding_cost=1, supply_type='U')
# Build the network.
network2 = SupplyChainNetwork()
network2.add_node(node3)
network2.add_successor(node3, node1)
network2.add_successor(node3, node2)
network2.add_predecessor(node1, node4)
# Check that network2 equals the one we built using network_from_edges().
network1.deep_equal_to(network2)

True

Instead of adding the nodes by successor, one can also add the nodes directly to the network and then add the edges in a separate step:


In [18]:
network3 = SupplyChainNetwork()
network3.add_node(node1)
network3.add_node(node2)
network3.add_node(node3)
network3.add_node(node4)
network3.add_edges_from_list([(3, 2), (3, 1), (4, 1)])

Another way to build `SupplyChainNetwork`s is using Stockpyl's built-in instances. The `instances` module contains approximately 70 instances, most (but not all) of which are taken from _FoSCT_ and named accordingly. The instances can be loaded using the [`load_instance()`](https://stockpyl.readthedocs.io/en/latest/api/other/instances.html#stockpyl.instances.load_instance) function by providing the instance name. A list of the built-in instances is provided in the online documentation for the \href[`instances`](https://stockpyl.readthedocs.io/en/latest/api/other/instances.html) module.

**Example:** Build the "digital camera" network from Figure 6.14 in _FoSCT_.


<center><img src="https://github.com/LarrySnyder/stockpyl/raw/master/notebooks/images/tutorial_digital_camera_system.png" width="600"/></center>

In [19]:
from stockpyl.instances import load_instance
instance = load_instance('figure_6_14')

This is equivalent to building the network manually:


In [20]:
from stockpyl.supply_chain_network import SupplyChainNetwork
from stockpyl.supply_chain_node import SupplyChainNode
from stockpyl.demand_source import DemandSource
from scipy import stats
instance2 = SupplyChainNetwork()
instance2.add_node(SupplyChainNode(1, 'Raw_Material', instance2,
	processing_time=2, local_holding_cost=0.01))
instance2.add_node(SupplyChainNode(2, 'Process_Wafers', instance2,
	processing_time=3, local_holding_cost=0.03))
instance2.add_node(SupplyChainNode(3, 'Package_Test_Wafers', instance2,
	processing_time=2, local_holding_cost=0.04))
instance2.add_node(SupplyChainNode(4, 'Imager_Base', instance2,
	processing_time=4, local_holding_cost=0.06))
instance2.add_node(SupplyChainNode(5, 'Imager_Assembly', instance2,
	processing_time=2, local_holding_cost=0.12))
instance2.add_node(SupplyChainNode(6, 'Ship_to_Final_Assembly', instance2,
	processing_time=3, local_holding_cost=0.13))
instance2.add_node(SupplyChainNode(7, 'Camera', instance2,
	processing_time=6, local_holding_cost=0.20))
instance2.add_node(SupplyChainNode(8, 'Circuit_Board', instance2,
	processing_time=4, local_holding_cost=0.08))
instance2.add_node(SupplyChainNode(9, 'Other_Parts', instance2,
	processing_time=3, local_holding_cost=0.04))
instance2.add_node(SupplyChainNode(10, 'Build_Test_Pack', instance2,
	processing_time=2, local_holding_cost=0.50, external_outbound_cst=2,
	demand_source=DemandSource(type='N', mean=0, standard_deviation=10)))
for n in instance2.nodes:
    n.demand_bound_constant = stats.norm.ppf(0.95)
instance2.add_edges_from_list([(1, 2), (2, 3), (3, 5), (4, 5), (5, 6),
	(7, 10), (6, 10), (8, 10), (9, 10)])
# Check that the two instances are equal.
instance.deep_equal_to(instance2)

True

---
**Exercise:** Write code to load the network from _FoSCT_ Example 6.5, pictured below, as a named instance. (The instance is called `example_6_5`.)

In the figure, the numbers inside the nodes are node indices; the numbers above each node are local holding costs; and the numbers below each node are processing times. The number on the inbound arrow to node 1 indicates the "external inbound CST", and those on the outbound arros from nodes 2 and 4 are "external outbound CSTs". It doesn't matter for now what those terms mean (we'll discuss them more below).

<center><img src="https://github.com/LarrySnyder/stockpyl/raw/master/notebooks/images/tutorial_example_6_5.png" width="500"/></center>


In [21]:
## SOLUTION
instance = load_instance('example_6_5')

Now build the network manually, and check that the two networks are equal.

(_Note_: Processing times are indicated using the keyword `processing_time`. External inbound and outbound CSTs are indicated using the keywords `external_inbound_cst` and `external_outbound_cst`.)



In [22]:
## SOLUTION
from stockpyl.supply_chain_network import network_from_edges
instance2 = network_from_edges(
    edges=[(1, 3), (3, 2), (3, 4)],
    node_order_in_lists=[1, 2, 3, 4],
    processing_time=[2, 1, 1, 1],
    external_inbound_cst=[1, None, None, None],
    local_holding_cost=[1, 3, 2, 3],
    demand_bound_constant=[1, 1, 1, 1],
    external_outbound_cst=[None, 0, None, 1],
    demand_type=[None, 'N', None, 'N'],
    mean=[None, 0, None, 0],
    standard_deviation=[None, 1, None, 1]
)
# Check that they are equal.
instance.deep_equal_to(instance2)

True

## 4.2 Local vs. Echelon Quantities

In a serial system indexed $N \to N-1 \to \cdots \to 1$, the _echelon_ of stage $j$ is defined as stage $j$ and all stages downstream from $j$. The _echelon inventory level_, $IL_j$, consists of all items on hand in stages $j,\ldots,1$, plus all items in transit between those stages, minus backorders at stage 1. Echelon-based quantities such as these are less intuitive than the _local_ quantities we typically deal with, but they are more convenient mathematically, and the functions described in Section 4.3 use them.

The _local holding cost_ at stage $j$, denoted $h'_j$, is the holding cost charged based on on-hand inventory at stage $j$ (sometimes plus items in transit from stage $j$ to $j-1$). This type of holding cost is intuitive, and is analogous to the holding cost in single-stage problems. On the other hand, in SSM models, it is often convenient to work with the _echelon holding cost_, denoted $h_j$ and defined as
$$h_j = h'_j - h'_{j+1},$$
with $h'_{N+1} \equiv 0$. One way to think about the echelon holding cost is that it represents the additional holding cost attributed to the value added to the product at stage $j$. Typically, echelon holding costs are non-negative (meaning the value of the product, or the cost to store it, increases as we move downstream), since local holding costs tend to increase as we move downstream. However, negative echelon holding costs are possible.
Local holding costs can be calculated from the echelon costs as follows:    
$$h'_j = \sum_{i=j}^N h_j.$$

In an _echelon base-stock policy_, each stage $j$ places an order so that its _echelon inventory position_ (defined as its echelon inventory level, $IL_j$, plus any items on-order from stage $j+1$) equals its _echelon base-stock level_, $S_j$. Echelon base-stock policies are closely related to the more familiar _local base-stock policies_, in which we bring the local inventory position up to the local base-stock level. Indeed, the two types of policies are equivalent if we relate the echelon base-stock levels $S_j$ and the local base-stock levels $S'_j$ as follows:
\begin{align*}
    S'_j & = S_j - S_{j-1} \\
    S_j & = \sum_{i=1}^j S'_i.
\end{align*}
We define $S_0 \equiv 0$. This assumes that $S_j \ge S_{j-1}$; if not, we let $S^-_j = \min_{i \ge j} \{S_i\}$ and set $S'_j = S^-_j - S^-_{j-1}$.

The serial SSM functions described in Section 4.3 take echelon holding costs as input parameters and return echelon base-stock levels as outputs. GSM models tend to work with local quantities, not echelon ones, so the functions described in Section 4.4 take local holding costs and return local base-stock levels. The functions [`local_to_echelon_base_stock_levels()`](https://stockpyl.readthedocs.io/en/latest/api/datatypes/supply_chain_network.html#stockpyl.supply_chain_network.local_to_echelon_base_stock_levels) and [`echelon_to_local_base_stock_levels()`](https://stockpyl.readthedocs.io/en/latest/api/datatypes/supply_chain_network.html#stockpyl.supply_chain_network.echelon_to_local_base_stock_levels) in the `supply_chain_network` module convert back and forth between the two types of base-stock levels. They take a dictionary in which the keys are node indices and the values are base-stock levels.

**Example:** The serial system below is from Example 6.1 in _FoSCT_. Suppose the local base-stock levels are $(S_1',S_2',S_3') = (10, 14, 25)$. The code below builds the network and then uses `local_to_echelon_base_stock_levels()` to translate the local base-stock levels into echelon ones.

<center><img src="https://github.com/LarrySnyder/stockpyl/raw/master/notebooks/images/tutorial_example_6_1.png" width="500"/></center>



In [30]:
from stockpyl.supply_chain_network import local_to_echelon_base_stock_levels
# Build the network.
network = serial_system(
    num_nodes=3,
    node_order_in_system=[3, 2, 1]
)
# Translate local to echelon base-stock levels.
S_local = {1: 10, 2: 14, 3: 25}
S_ech = local_to_echelon_base_stock_levels(network, S_local)
S_ech

{3: 49, 2: 24, 1: 10}

In [31]:
from stockpyl.supply_chain_network import echelon_to_local_base_stock_levels
# Translate back to local.
echelon_to_local_base_stock_levels(network, S_ech)

{3: 25, 2: 14, 1: 10}

## 4.3 Serial SSM Systems

The `ssm_serial` module contains code to solve serial systems under the stochastic service model (SSM), either exactly, using the [`optimize_base_stock_levels()`](https://stockpyl.readthedocs.io/en/latest/api/meio/ssm_serial.html#stockpyl.ssm_serial.optimize_base_stock_levels) function (which implements the algorithm by Chen and Zheng (1994), which in turn is based on the algorithm by Clark and Scarf (1960)), or approximately, using the [`newsvendor_heuristic()`](https://stockpyl.readthedocs.io/en/latest/api/meio/ssm_serial.html#stockpyl.ssm_serial.newsvendor_heuristic) function (which implements the newsvendor-based heuristic by Shang and Song 2003). Both functions assume a continuous-review system in which each stage follows an echelon base-stock policy (see Section 4.2).

The `optimize_base_stock_levels()` function has the signature
```python
optimize_base_stock_levels(num_nodes=None, node_order_in_system=None,
	node_order_in_lists=None, echelon_holding_cost=None, lead_time=None,
	stockout_cost=None, demand_mean=None, demand_standard_deviation=None,
	demand_source=None, network=None, S=None, plots=False, x=None,
	x_num=1000, d_num=100, ltd_lower_tail_prob=1-stats.norm.cdf(4),
	ltd_upper_tail_prob=1-stats.norm.cdf(4),
	sum_ltd_lower_tail_prob=1-stats.norm.cdf(4),
	sum_ltd_upper_tail_prob=1-stats.norm.cdf(4))
```
and returns two parameters: `S_star` is a dictionary containing the optimal echelon base-stock levels, and `C_star` is the optimal expected cost.

The `newsvendor_heuristic()`  function has the signature
```python
newsvendor_heuristic(num_nodes=None, node_order_in_system=None,
    node_order_in_lists=None, echelon_holding_cost=None, lead_time=None,
    stockout_cost=None, demand_mean=None, demand_standard_deviation=None,
    demand_source=None, network=None, weight=0.5, round_type=None)
```
and returns a single parameter, `S_heur`.

For either function, you may pass the instance data as individual parameters (costs, demand distribution, etc.) or a `SupplyChainNetwork`.

By default, the nodes in the system are assumed to be indexed $N,\ldots,1$, with node 1 at the downstream end, but this can be changed by providing either the `node_order_in_system` parameter or by providing a `SupplyChainNetwork` explicitly in the `network` parameter.

The node-based parameters `echelon_holding_cost` and `lead_time` can be specified either as a dictionary, a list, or a singleton. The requirements for specifying these parameters are the same as those described in Section 4.1, except that the default indexing of the nodes for `ssm_serial` functions is 1,...,`num_nodes` instead of 0,...,`num_nodes`-1.

The easiest way to describe the demand is to provide the `demand_mean` and `demand_standard_deviation` parameters; in this case, the demand is assumed to be normally distributed. Alternatively, you can provide a [`DemandSource`](https://stockpyl.readthedocs.io/en/latest/api/datatypes/demand_source.html#stockpyl.demand_source.DemandSource) object in the `demand_source` parameter, which is a bit more work but is more flexible. Finally, if you pass the whole network as a `SupplyChainNetwork` object in the `network` parameter, you can specify the demand distribution in that object.





**Example:** Here is Example 6.1 from _FoSCT_ with the data passed as individual parameters. In this 3-stage instance, the nodes are indexed $3 \to 2 \to 1$. The demand at stage 1 is distributed as $N(5,1^2)$ per unit time. The lead times are $L_1=L_2=1$ and $L_3=2$. Echelon holding costs are $(h_1,h_2,h_3) = (3,2,2)$. The stockout cost at stage 1 is $p=37.12$ per unit time. We solve it using the exact algorithm implemented in the `optimize_base_stock_levels()` function (see Section 4.3.2 below).

In [32]:
from stockpyl.ssm_serial import optimize_base_stock_levels
# Optimize, passing the data as individual parameters.
S_star, C_star = optimize_base_stock_levels(
    num_nodes=3,
    echelon_holding_cost=[2, 2, 3],
    lead_time=[2, 1, 1],
    stockout_cost=37.12,
    demand_mean=5,
    demand_standard_deviation=1
)
# Display optimal base-stock levels.
S_star

{3: 22.700237234889784, 2: 12.012332294949644, 1: 6.5144388073261155}

In [33]:
# Display optimal expected cost.
C_star

47.668653127136345

And here is the same example, first building a `SupplyChainNetwork` and then passing that instead:

In [34]:
# Build network object.
example_6_1_network = serial_system(
    num_nodes=3,
    node_order_in_system=[3, 2, 1],
    echelon_holding_cost={1: 3, 2: 2, 3: 2},
    shipment_lead_time={1: 1, 2: 1, 3: 2},
    stockout_cost={1: 37.12, 2: 0, 3: 0},
    demand_type='N',
    mean=5,
    standard_deviation=1,
    policy_type='BS',
    base_stock_level=0,
	)
# Pass object and optimize.
S_star, C_star = optimize_base_stock_levels(network=example_6_1_network)
S_star


{3: 22.700237234889784, 2: 12.012332294949644, 1: 6.5144388073261155}

Example 6.1 is also a built-in instance in Stockpyl, so you can load it directly:


In [35]:
S_star, C_star = optimize_base_stock_levels(
    network=load_instance("example_6_1")
)
S_star

{3: 22.700237234889784, 2: 12.012332294949644, 1: 6.5144388073261155}

The code snippets above use the exact optimization algorithm. We can instead solve the instance using the newsvendor-based heuristic (see Section 4.3.3 below):

In [36]:
from stockpyl.ssm_serial import newsvendor_heuristic
# Solve using newsvendor-based heuristic.
S_heur = newsvendor_heuristic(network=example_6_1_network)
S_heur

{3: 22.634032391786285, 2: 12.027434723327854, 1: 6.490880975286938}

In [37]:
# Evaluate the (exact) expected cost of the heuristic solution.
from stockpyl.ssm_serial import expected_cost
expected_cost(S_heur, network=example_6_1_network)

47.68009914084217

---
**Exercise:**  Consider the 5-node serial system in _FoSCT_ Problem 6.2(a). In this instance, the nodes are indexed $5\to \cdots \to 1$. The echelon holding costs are $h_1=h_2=2$ and $h_3=h_4=h_5=1$. The lead times are $L_1=\cdots=L_5=0.5$. The stockout cost is $p=24$. Optimize this system, both exactly and heuristically, and compare the resulting expected costs.

In [40]:
## SOLUTION
# Build network.
instance = load_instance('problem_6_2a')
# Optimize exactly.
S_star, C_star = optimize_base_stock_levels(network=instance)
S_star

{5: 174.4384998006611,
 4: 142.02349425247698,
 3: 108.98512321298162,
 2: 74.38833844520818,
 1: 40.41491916874597}

In [41]:
## SOLUTION
C_star

451.5679149371823

In [42]:
## SOLUTION
# Optimize heuristically.
S_heur = newsvendor_heuristic(network=instance)
S_heur

{5: 175.83298580667355,
 4: 142.8985667640439,
 3: 109.59625626575588,
 2: 74.45806987058583,
 1: 40.58670401687929}

In [43]:
## SOLUTION
expected_cost(S_heur, network=instance)

451.8055803098469

## 4.4 Serial or Tree GSM Systems



### 4.4.1 Overview and Assumptions

GSM models assume a periodic-review system in which each stage follows a local base-stock policy. In GSM systems, we assume that each stage offers a _committed service time_ (CST) to its customers (which may be downstream stages or external demands) within which it promises to satisfy each order. The CSTs are the decision variables in the optimization model. To ensure that each stage meets its CST, the GSM model assumes that the external demands are bounded. In particular, if the demands in one time period are distributed as $N(\mu,\sigma^2)$, then we assume that the total demand in any $\tau$ time periods is less than or equal to
$$D(\tau) = \mu\tau + z_\alpha\sigma\sqrt{\tau}$$
for some constant $\alpha$. If the demand in a given $\tau$-period interval exceeds $D(\tau)$, the excess demands are assumed to be handled outside the system, e.g., by outsourcing.

> **Remark:** It is tempting to set $\alpha$ to something large so that the demand bound is quite loose and the demand model is more accurate. However, as we will see below, the base-stock levels are directly related to $\alpha$, so setting $\alpha$ to a large value will also result in unnecessarily large base-stock levels and resulting holding costs. This tension is one of the main disadvantages of the GSM approach.

The GSM approach was first discussed by Kimball (1988). Simpson (1958) applied it to serial systems, and Inderfurth (1991) (see also Graves 1988) discussed how to solve it using dynamic programming (DP); this is the basis for the approach implemented in the [`gsm_serial`](https://stockpyl.readthedocs.io/en/latest/api/meio/gsm_serial.html) module. For pure assembly and distribution systems, see Inderfurth (1991), Minner (1997), and Inderfurth and Minner (1998). Graves and Willems (2000) present a DP model for tree systems, that is, networks that have no directed cycles; this is the algorithm implemented in the [`gsm_tree`](https://stockpyl.readthedocs.io/en/latest/api/meio/gsm_tree.html) module. Magnanti, et al. (2006) present a MIP-based model for solving general systems.

### 4.4.2 Serial Systems

First consider a serial network indexed $N \to \cdots \to 1$. At each stage $i$, let $T_i$ be the _processing time_ at $i$, representing the number of periods required to perform the activities at the stage. (GSM models use processing times $T$ at the nodes rather than lead times $L$ on the edges.) The $T_i$ are constants. Furthermore, let $S_i$ be the _outbound CST_ that stage $i$ promises its customers, and let $SI_i$ be the _inbound CST_ that stage $i$ receives from its suppliers. Note that the inbound CST at $j$ is simply the outbound CST at its predecessor.


Node 1 faces external demand and has a CST of $s_1$; this is a constant, not a decision variable, representing the contract agreed to by the firm and the external customer. Similarly, stage $N$ has a fixed inbound CST $si_N$ from its (external) supplier. The optimization problem finds $S_i$ for $2 \le i \le N$.


> **Remark:** Unfortunately, the SSM literature has tended to use $S$ to represent a base-stock level, while the GSM literature has tended to use $S$ to represent a CST. We will stay consistent with these two bodies of literature, which means using $S$ in two different ways in Section 4.3 vs. Section 4.4

The quantity $SI_i + T_i - S_i$ is called the _net lead time_; it represents the number of periods that stage $i$'s inventory must cover. In particular, if stage $i$ uses a (local) base-stock level of
$$y_i = \mu(SI_i + T_i - S_i) + z_\alpha\sigma\sqrt{SI_i + T_i - S_i}$$
then the stage will always be able to meet its CST.

> **Remark:** The equation above gives us a one-to-one relationship between the CSTs and the base-stock levels. Although the CSTs are the decision variables in a GSM model, it is usually more intuitive to convert these to base-stock levels for managerial purposes.



The [`optimize_committed_service_times()`](https://stockpyl.readthedocs.io/en/latest/api/meio/gsm_serial.html#stockpyl.gsm_serial.optimize_committed_service_times) function in the `gsm_serial` module implements the DP by Inderfurth (1991) for serial GSM systems. It has the following signature:
```python
optimize_committed_service_times(num_nodes=None, local_holding_cost=None,
	processing_time=None, demand_bound_constant=None,
	external_outbound_cst=None, external_inbound_cst=None,
	demand_mean=None, demand_standard_deviation=None, demand_source=None,
	network=None)
```
It returns two parameters, `opt_cst` and `opt_cost`.


You may specify the network either by its individual parameters (`num_nodes`, `local_holding_cost`, etc.)\ or in the `network` parameter as a `SupplyChainNetwork`. If you specify individual parameters, the nodes of the network must be indexed $N \to \cdots \to 1$. (If you specify it in the `network` parameter, the nodes may be indexed in any way.) The node-specific parameters (`local_holding_cost`, `processing_time`, and `demand_bound_constant`) may be a dictionary, a list, or a singleton, with the following requirements:
* If the parameter is a dictionary, its keys must equal 1,...,`num_nodes`, each corresponding to a node index.
* If the parameter is a list, it must have length `num_nodes`; the `n`th entry in the list corresponds to the node with index `n`+1.
* If the parameter is a singleton, all nodes will have that parameter set to the singleton value.

These requirements are less flexible than those for the `ssm_serial` module (see Section 4.3.1), but you can always build a `SupplyChainNetwork` in whatever way you like, and then pass it in the `network` parameter, which offers full flexibility.

You must either pass the `demand_mean` and `demand_parameters`, or pass a [`DemandSource`](https://stockpyl.readthedocs.io/en/latest/api/datatypes/demand_source.html#stockpyl.demand_source.DemandSource) object in the `demand_source` parameter.

**Example:** _FoSCT_ Example 6.3 is shown below. The numbers below the stages are the processing times $T_i$. The number on the inbound arrow to stage 3 indicates that $si_3=1$, while the outbound number from stage 1 indicates that the fixed CST $s_1 = 1$.  The holding costs at stages 1, 2, and 3 are 7, 4, and 2, respectively, and are noted above each stage.  Finally, $z_\alpha=\sigma_i=1$ at all stages.


<center><img src="https://github.com/LarrySnyder/stockpyl/raw/master/notebooks/images/tutorial_example_6_3.png" width="500"/></center>


In [44]:
from stockpyl.gsm_serial import optimize_committed_service_times
opt_cst, opt_cost = optimize_committed_service_times(
    num_nodes=3,
    local_holding_cost=[7, 4, 2],
    processing_time=[1, 0, 1],
    demand_bound_constant=1,
    external_outbound_cst=1,
    external_inbound_cst=1,
    demand_mean=0,
    demand_standard_deviation=1
)
opt_cst

{3: 0, 2: 0, 1: 1}

In [45]:
opt_cost

2.8284271247461903

Alternatively, we can pass a `SupplyChainNetwork`:

In [46]:
from stockpyl.supply_chain_network import network_from_edges
example_6_3_network = network_from_edges(
    edges=[(3, 2), (2, 1)],
    node_order_in_lists=[1, 2, 3],
    processing_time=[1, 0, 1],
    external_inbound_cst=[None, None, 1],
    local_holding_cost=[7, 4, 2],
    demand_bound_constant=1,
    external_outbound_cst=[1, None, None],
    demand_type=['N', None, None],
    mean=0,
    standard_deviation=[1, 0, 0]
	)
optimize_committed_service_times(network=example_6_3_network)

({3: 0, 2: 0, 1: 1}, 2.8284271247461903)

Or load the instance directly:

In [47]:
optimize_committed_service_times(network=load_instance("example_6_3"))

({3: 0, 2: 0, 1: 1}, 2.8284271247461903)

**Exercise:** Consider a 3-stage serial system indexed $3\to 2\to 1$. The processing times at stages 1, 2, and 3 are 3, 8, and 2, respectively. There is an external inbound CST of $si_3=4$ and an external outbound CST of $s_1=2$. The holding costs at stages 1, 2, and 3 are 40, 30, and 10, respectively. The demand bound uses $z_\alpha=2$ at all stages. The demand standard deviation is $\sigma=10$. Optimize this system.

In [50]:
## SOLUTION
opt_cst, opt_cost = optimize_committed_service_times(
    num_nodes=3,
    local_holding_cost=[40, 30, 10],
    processing_time=[3, 8, 2],
    demand_bound_constant=2,
    external_outbound_cst=2,
    external_inbound_cst=4,
    demand_mean=0,
    demand_standard_deviation=10
)
opt_cst

{3: 0, 2: 8, 1: 2}

In [52]:
## SOLUTION
opt_cost

2889.8979485566356

### 4.4.3 Tree Systems

Now consider a tree system, i.e., a network with no directed cycles. Let $A$ be the set of arcs (edges) in the network. The network may have multiple demand stages (stages that face external demand); each faces $N(\mu_i,\sigma_i^2)$ demand per period and has a fixed outbound CST of $s_i$. Similarly, it may have multiple supply stages (stages that receive external supply); each has a fixed inbound CST of $si_i$.

Stages that are not demand stages see demand that is derived from their successor stages. The resulting demand is normally distributed with mean and standard deviation
	\begin{align}
		\mu_i & = \sum_{(i,j)\in A} \mu_j \\
		\sigma_i & = \sqrt{\sum_{(i,j)\in A} \sigma_j^2}.
	\end{align}
If stage $i$ has more than one successor, we assume that it quotes the same CST to each of them. If stage $i$ has more than one predecessor, then
$$SI_i = \max_{(j,i)\in A} \{S_j\},$$
since stage $i$ requires all predecessor units to be present before it begins processing.

The relationship between the CSTs and the base-stock levels is the same as in the serial case.
The [`optimize_committed_service_times()`](https://stockpyl.readthedocs.io/en/latest/api/meio/gsm_tree.html#stockpyl.gsm_tree.optimize_committed_service_times) function in the `gsm_tree` module implements the DP by Graves and Willems (2000) for tree GSM systems. This DP is considerably more complicated than the one for serial systems, so we omit a detailed discussion here.

The `optimize_committed_service_times()` has the following signature:
```python
optimize_committed_service_times(tree)
```
It returns two parameters, `opt_cst` and `opt_cost`. This function does not allow you to specify the instance as individual parameters; instead, you must provide a `SupplyChainNetwork` object in the `tree` parameter.

**Example:** The 4-node system shown below is from Example 6.5 in _FoSCT_.  The numbers below the stages are the processing times $T_i$.  The number on the inbound arrow to stage 1 indicates that $si_1=1$, while the outbound numbers from stages 2 and 4 indicate the fixed CSTs $s_i$.  The holding costs are noted above each stage and are equal to 1 at stage 1, 2 at stage 3, and 3 at stages 2 and 4.  At all stages, $z_\alpha=1$ and $\sigma_2=\sigma_4=1$; then $\sigma_1=\sigma_3=\sqrt{2}$. The following code snippet solves this instance:

<center><img src="https://github.com/LarrySnyder/stockpyl/raw/master/notebooks/images/tutorial_example_6_5.png" width="500"/></center>

In [54]:
from stockpyl.gsm_tree import optimize_committed_service_times
example_6_5_network = network_from_edges(
    edges=[(1, 3), (3, 2), (3, 4)],
    node_order_in_lists=[1, 2, 3, 4],
    processing_time=[2, 1, 1, 1],
    external_inbound_cst=[1, None, None, None],
    local_holding_cost=[1, 3, 2, 3],
    demand_bound_constant=[1, 1, 1, 1],
    external_outbound_cst=[None, 0, None, 1],
    demand_type=[None, 'N', None, 'N'],
    mean=0,
    standard_deviation=[None, 1, None, 1]
	)
opt_cst, opt_cost = optimize_committed_service_times(tree=example_6_5_network)
opt_cst


{1: 0, 3: 0, 2: 0, 4: 1}

In [55]:
opt_cost

8.277916867529369

Example 6.5 is a built-in instance in Stockpyl, so it can be loaded directly instead:


In [56]:
optimize_committed_service_times(tree=load_instance("example_6_5"))

({1: 0, 3: 0, 2: 0, 4: 1}, 8.277916867529369)

---
**Exercise:** Solve the instance pictured below (_FoSCT_ Problem 6.9).

<center><img src="https://github.com/LarrySnyder/stockpyl/raw/master/notebooks/images/tutorial_problem_6_9.png" width="500"/></center>

In [57]:
## SOLUTION
optimize_committed_service_times(tree=load_instance("problem_6_9"))

({1: 3, 2: 3, 3: 0, 4: 0, 5: 0, 6: 2}, 15.649530249501026)

## 4.5 General MEIO Systems

For MEIO systems with arbitrarily topology (not necessarily serial or tree systems), the [`meio_general`](https://stockpyl.readthedocs.io/en/latest/api/meio/meio_general.html#module-stockpyl.meio_general) module can optimize base-stock levels approximately using relatively brute-force approaches—either coordinate descent or enumeration. These heuristics tend to be quite slow and not particularly accurate, but they are sometimes the best methods available for complex systems that are not well solved in the literature.

For both approaches, you may provide an objective function that will be used to evaluate each candidate solution, or you may omit the objective function and the algorithm will evaluate solutions using simulation. Obviously, evaluating using simulation is typically much slower than using an objective function. (See the [`online documentation`](https://stockpyl.readthedocs.io/en/latest/tutorial/tutorial_sim.html) for more information about Stockpyl's simulation features.)

Both approaches assume that each stage follows a base-stock policy, and they attempt to optimize the base-stock levels. (Future versions of Stockpyl may allow a wider range of policies.) Both functions use local, not echelon, costs and base-stock levels.

The [`meio_by_enumeration()`](https://stockpyl.readthedocs.io/en/latest/api/meio/meio_general.html#stockpyl.meio_general.meio_by_enumeration) function  optimizes an MEIO instance by enumerating combinations of values of the base-stock levels. It has signature
```python
meio_by_enumeration(network, base_stock_levels=None, truncation_lo=None,
	truncation_hi=None, discretization_step=None, discretization_num=None,
	groups=None, objective_function=None, sim_num_trials=10,
	sim_num_periods=1000, sim_rand_seed=None, progress_bar=True,
	print_solutions=False)
```
and returns two parameters, `best_S` (the best base-stock levels found) and `best_cost` (the corresponding expected cost).

The function provides several ways to control how the enumeration is performed, via optional input parameters. In particular:

* `base_stock_levels` is a dictionary indicating, for each node index, the base-stock levels to test in the enumeration.
* `truncation_lo` and `truncation_hi` indicate the low and high ends of the truncation range for base-stock levels to test for each node. You can specify either a singleton (in which case every node uses the same value) or a dictionary whose keys are the node indices.
* `discretization_step` indicates the interval width to use for discretizing the base-stock levels to test for each node. The `discretization_num` parameter indicates how many intervals to use. Only one should be provided. Either can be a dictionary or singleton.
* `groups` is a list of sets, each of which contains indices of nodes that should have the same base-stock level. This speeds the optimization since the base-stock levels for the nodes in a given group do not have to be optimized individually. Any nodes not contained in any set in the list are optimized individually.
* `objective_function` is the function to use to evaluate a given solution. The function must take a single argument, the dictionary of base-stock levels, and return a single output, the expected cost per period. If this parameter is omitted, simulation will be used instead to evaluate solutions.
* `sim_num_trials`, `sim_num_periods`, and `sim_rand_seed` can be used to customize the simulation used to evaluate solutions.
* `progress_bar` specifies whether a progress bar is displayed during the search.
* `print_solutions` specifies whether to display each solution and its cost during the search.



**Example:** In the code snippet below, we solve the 3-node serial SSM system in Example 6.1 from _FoSCT_ using enumeration. We specify upper and lower bounds on the base-stock levels to test for each node and evaluate each candidate set of base-stock levels using simulation (3 trials, 100 periods per trial—a very coarse approximation since the simulation runs are very small).

In [58]:
from stockpyl.meio_general import meio_by_enumeration
example_6_1_network = load_instance("example_6_1")
best_S, best_cost = meio_by_enumeration(
    network=example_6_1_network,
    truncation_lo={1: 5, 2: 4, 3: 10},
    truncation_hi={1: 7, 2: 7, 3: 12},
    sim_num_trials=3,
    sim_num_periods=100,
    sim_rand_seed=42
	)
best_S

100%|██████████| 36/36 [00:11<00:00,  3.18it/s]


{3: 11, 2: 5, 1: 7}

In [59]:
best_cost

47.99409584298761

Recall that the optimal local base-stock levels are 6.51, 5.50, and 10.69 (converting from the echelon base-stock levels in Section 4.3.1), with optimal cost 47.69. Therefore, the solution found by enumeration is quite good—it is only 0.7\% worse than optimal. On the other hand, we stacked the deck by giving the function a pretty narrow range of base-stock levels to test. For a fairer experiment, we would test a broader range of base-stock levels, but then the execution would be even slower.

Alternatively, we can provide an objective function. This is more accurate and faster than evaluating solutions using simulation, but if the objective function must be evaluated numerically (as it does for serial SSM systems), speed and accuracy are still non-trivial issues to consider. In the code below, we first define an objective function using a Python lambda function; it evaluates each solution by first converting the local base-stock levels to echelon and then passing them to the `expected_cost()` function for serial SSM systems, which requires echelon base-stock levels as inputs. The discretization settings used below (`x_num`=100, `d_num`=10) are relatively coarse, producing inaccurate solutions but fairly quickly.


In [60]:
from stockpyl.ssm_serial import expected_cost
from stockpyl.supply_chain_network import local_to_echelon_base_stock_levels
obj_fcn = lambda S: expected_cost(local_to_echelon_base_stock_levels(
		example_6_1_network, S), network=example_6_1_network, x_num=100, d_num=10
	)
best_S, best_cost = meio_by_enumeration(
    network=example_6_1_network,
    truncation_lo={1: 5, 2: 4, 3: 10},
    truncation_hi={1: 7, 2: 7, 3: 12},
    objective_function=obj_fcn
	)
best_S

100%|██████████| 36/36 [00:05<00:00,  6.47it/s]


{3: 11, 2: 5, 1: 7}

In [61]:
best_cost

48.21449789525488

The [`meio_by_coordinate_descent()`](https://stockpyl.readthedocs.io/en/latest/api/meio/meio_general.html#stockpyl.meio_general.meio_by_coordinate_descent) function optimizes (approximately) using coordinate descent. In principle, coordinate descent will find the globally optimal solution if the objective function is jointly convex in the base-stock levels, but if solutions are evaluated using simulation, then there are no guarantees. Like enumeration, coordination can be quite slow and not particularly accurate.

The function's signature is
```python
meio_by_coordinate_descent(network, initial_solution=None,
	search_lo=None, search_hi=None, groups=None, objective_function=None,
	sim_num_trials=10, sim_num_periods=1000, sim_rand_seed=None, tol=1e-2,
	line_search_tol=1e-4, verbose=False)
```
It returns two parameters, `best_S` and `best_cost`. The optional input parameters give you control over how the search is performed. Several are identical to those for `meio_by_enumeration()` (see above). Others include:
* `initial_solution` is the starting solution for the coordinate descent search, specified as a dictionary whose keys are node indices and whose values are base-stock levels. If omitted, the initial solution will be set automatically.
* `search_lo` and `search_hi`  indicate the low and high ends of the search range for the base-stock levels at each node. You can specify either a singleton (in which case every node uses the same value) or a dictionary whose keys are the node indices.
* `tol` is a tolerance; the algorithm terminates when an iteration fails to improve the objective function by more than `tol`.
* `line_search_tol` is a tolerance to use for the line search (golden section search) component of the coordinate descent algorithm.
* `verbose` indicates whether messages should be displayed at each iteration.


**Example:** In the following code snippets, we again solve _FoSCT_ Example 6.1, this time using coordinate descent. First, we use simulation.

(_Note:_ These functions are relatively slow. Please be patient!)

In [62]:
from stockpyl.meio_general import meio_by_coordinate_descent
from stockpyl.ssm_serial import expected_cost
from stockpyl.supply_chain_network import local_to_echelon_base_stock_levels
best_S, best_cost = meio_by_coordinate_descent(
    network=example_6_1_network,
    search_lo={1: 5, 2: 4, 3: 10},
    search_hi={1: 7, 2: 7, 3: 12},
    sim_num_trials=3,
    sim_num_periods=100,
    sim_rand_seed=762
	)
best_S

{3: 10.00004085634901, 2: 5.818278530757848, 1: 6.516684268315169}

In [63]:
best_cost

46.90498842606734

This time we provide an objective function:

In [64]:
obj_fcn = lambda S: expected_cost(local_to_echelon_base_stock_levels(
	example_6_1_network, S), network=example_6_1_network, x_num=20, d_num=10
)
best_S, best_cost = meio_by_coordinate_descent(
    example_6_1_network,
    search_lo={1: 5, 2: 4, 3: 10},
    search_hi={1: 7, 2: 7, 3: 12},
    objective_function=obj_fcn
)
best_S

{3: 10.995764600780891, 2: 6.887792096395302, 1: 6.004184897994424}

In [65]:
best_cost

62.33491040676202

---
**Exercise:** Consider a 2-node serial system ($2\to 1$) in which $p=15$, $L_1=L_2=1$, $h_1=h_2=1$, and the demand per unit time is distributed as $N(100,15^2)$ (_FoSCT_ Problem 6.1). Solve this instance in three ways:

* exactly, using `optimize_base_stock_levels()`
* approximately, using `meio_by_enumeration()` (either using simulation or providing an objective function)
* approximately, using `meio_by_coordinate_descent()` (either using simulation or providing an objective function)

Use the optimal solution to give the two approximate methods very tight bounds for the base-stock levels to search—it's cheating, but if you don't do it, you'll be waiting forever!

Compare the costs of the three solutions found. (_Note:_ `optimize_base_stock_levels()` and `expected_cost()` use echelon base-stock levels. `meio_by_enumeration()` and `meio_by_coordinate_descent()` return local base-stock levels.)



In [69]:
## SOLUTION
# Build the network.
network=load_instance("problem_6_1")
# Solve exactly.
S_star, C_star = optimize_base_stock_levels(network=network)
S_star, C_star

({2: 228.6691776877484, 1: 123.70746751331217}, 166.2322961200302)

In [78]:
## SOLUTION
# Convert to local base-stock levels.
echelon_to_local_base_stock_levels(network, S_star)

{2: 104.96171017443623, 1: 123.70746751331217}

In [79]:
## SOLUTION
# Solve using enumeration.
best_S_enum, best_cost_enum = meio_by_enumeration(
    network=network,
    truncation_lo={1: 120, 2: 100},
    truncation_hi={1: 125, 2: 110},
    sim_num_trials=3,
    sim_num_periods=100,
    sim_rand_seed=42
	)
best_S_enum

100%|██████████| 66/66 [00:09<00:00,  7.19it/s]


{2: 106, 1: 124}

In [80]:
## SOLUTION
expected_cost(local_to_echelon_base_stock_levels(network, best_S_enum), network=network)

166.36789622271175

In [84]:
## SOLUTION
# Solve using coordinate descent.
best_S_cd, best_cost_cd = meio_by_coordinate_descent(
    network=network,
    search_lo={1: 120, 2: 100},
    search_hi={1: 125, 2: 110},
    sim_num_trials=3,
    sim_num_periods=100,
    sim_rand_seed=762
	)
best_S_cd

{2: 102.27611646375085, 1: 123.58759475268724}

In [85]:
## SOLUTION
expected_cost(local_to_echelon_base_stock_levels(network, best_S_cd), network=network)

166.57406998070135